In [2]:
import pandas as pd
from transformers import pipeline
import os
import re

In [ ]:
def aplicar_sentimento(caminho_csv, pasta_saida='outputs'):
    print("\n--- FinBERT-PT-BR ---")
    df = pd.read_csv(caminho_csv)
    pipe = pipeline("text-classification", model="lucas-leme/FinBERT-PT-BR")

    sentimentos = df['texto'].apply(lambda x: pipe(x[:512])[0])
    
    # df['texto_limpo'] = df['texto'].apply(limpar_texto)
    # sentimentos = df['texto_limpo'].apply(lambda x: pipe(x[:512])[0])


    df['sentimento'] = sentimentos.apply(lambda s: s['label'])
    df['score_sentimento'] = sentimentos.apply(lambda s: s['score'])

    os.makedirs(pasta_saida, exist_ok=True)
    nome_saida = os.path.basename(caminho_csv).replace('.csv', '_sentimento.csv')
    caminho_saida = os.path.join(pasta_saida, nome_saida)
    df.to_csv(caminho_saida, index=False, encoding='utf-8')
    print("Sentimentos salvos em:", caminho_saida)

    # Sentimento geral
    score_total = 0
    num_total = 0

    for i, row in df.iterrows():
        label = row['sentimento']
        score = row['score_sentimento']
        if label == 'positive':
            score_total += score
            num_total += 1
        elif label == 'negative':
            score_total -= score
            num_total += 1
        elif label == 'neutral':
            # neutros como 0 no calculo
            score_total += 0
            num_total += 1

    if num_total == 0:
        print("Nenhuma notícia classificada.")
        return

    media = score_total / num_total
    polaridade = (
        "Positive" if media >= 0.15 else
        "Negative" if media <= -0.15 else
        "Neutral"
    )
    print(f"Sentimento geral: {polaridade} (Score médio: {media:.3f})")

In [ ]:
# def limpar_texto(texto):
#     # Remover chamadas promocionais comuns
#     texto = re.sub(r"(?i)(receba as notícias.*?whatsapp.*?)", "", texto)
#     texto = re.sub(r"(?i)e-mail inválido.*?(invest)?", "", texto)
    
#     # Remover datas/horários embutidos (opcional)
#     texto = re.sub(r"\d{1,2} de \w+ de \d{4}", "", texto)
#     texto = re.sub(r"\d{1,2}h\d{0,2}", "", texto)  # Ex: 20h31

#     # Remover múltiplos espaços e quebras de linha
#     texto = re.sub(r"\s+", " ", texto).strip()
    
#     return texto


In [4]:
if __name__ == '__main__':
    caminho = "C:\\Users\\femor\\OneDrive\\Área de Trabalho\\Analise de Sentimento\\Part01\\noticias"
    arquivos_csv = [f for f in os.listdir(caminho) if f.endswith('.csv')]

    for nome_arquivo in arquivos_csv:
        caminho_completo = os.path.join(caminho, nome_arquivo)
        print(f"\nProcessando arquivo: {nome_arquivo}")
        aplicar_sentimento(caminho_completo)


Processando arquivo: noticias_2025-05-01.csv

=== FinBERT-PT-BR ===


Device set to use cuda:0


Sentimentos salvos em: outputs\noticias_2025-05-01_sentimento.csv
Nenhuma notícia classificada.

Processando arquivo: noticias_2025-05-02.csv

=== FinBERT-PT-BR ===


Device set to use cuda:0


Sentimentos salvos em: outputs\noticias_2025-05-02_sentimento.csv
Nenhuma notícia classificada.

Processando arquivo: noticias_2025-05-03.csv

=== FinBERT-PT-BR ===


...

Sentimentos salvos em: outputs\noticias_2025-05-11_sentimento.csv
Nenhuma notícia classificada.

Processando arquivo: noticias_2025-05-12.csv

=== FinBERT-PT-BR ===


Device set to use cuda:0


Sentimentos salvos em: outputs\noticias_2025-05-12_sentimento.csv
Nenhuma notícia classificada.

Processando arquivo: noticias_2025-05-13.csv

=== FinBERT-PT-BR ===


Device set to use cuda:0


Sentimentos salvos em: outputs\noticias_2025-05-13_sentimento.csv
Nenhuma notícia classificada.


In [5]:
pasta_saida = 'outputs'

arquivos_saida = [f for f in os.listdir(pasta_saida) if f.endswith('_sentimento.csv')]

for arquivo in sorted(arquivos_saida):
    caminho_completo = os.path.join(pasta_saida, arquivo)
    print(f"\n--- Visualizando: {arquivo} ---")
    display(pd.read_csv(caminho_completo))


--- Visualizando: noticias_2025-05-01_sentimento.csv ---


,url,data,titulo,texto,fonte,sentimento,score_sentimento
0,https://exame.com/mundo/pacotes-vindos-da-chin...,2025-05-01,Pacotes vindos da China aos EUA sofrerão tarif...,"Pacotes provenientes da China, especialmente a...",Exame,NEGATIVE,0.838735
1,https://exame.com/invest/mercados/vendas-globa...,2025-05-01,Vendas globais do McDonald’s caem após tarifas...,Percepção negativa sobre os EUA e marcas ameri...,Exame,NEGATIVE,0.827320
2,https://exame.com/esg/o-ano-pos-tragedia-desas...,2025-05-01,O ano pós tragédia: desastres climáticos no RS...,Patrocínio: Parceiro institucional: O Rio Gran...,Exame,NEGATIVE,0.775204
3,https://exame.com/pop/cantora-nana-caymmi-morr...,2025-05-01,Cantora Nana Caymmi morre aos 84 anos,"Nana Caymmi, uma das grandes intérpretes da mú...",Exame,NEGATIVE,0.821119
4,https://exame.com/invest/mercados/microsoft-au...,2025-05-01,Microsoft aumenta preços do Xbox em meio à gue...,"Microsoft: em comunicado, a Xbox disse que aum...",Exame,NEGATIVE,0.786238
...,...,...,...,...,...,...,...
76,https://valor.globo.com/eu-e/noticia/2025/05/0...,2025-05-01,Eleição de Míriam Leitão para ABL marca trajet...,01/05/2025 13h24Atualizado01/05/2025 A jornali...,Valor Econômico,POSITIVE,0.665008
77,https://valor.globo.com/mundo/noticia/2025/05/...,2025-05-01,Dia do Trabalhador é marcado por protestos ant...,Por Valor— Rio 01/05/2025 17h45Atualizado01/05...,Valor Econômico,NEGATIVE,0.849405
78,https://valor.globo.com/empresas/noticia/2025/...,2025-05-01,"Microsoft sobe 9,2% e supera Apple como maior ...",01/05/2025 12h48Atualizado01/05/2025 As ações ...,Valor Econômico,POSITIVE,0.736993
79,https://valor.globo.com/financas/noticia/2025/...,2025-05-01,Petróleo fecha em forte alta com maior apetite...,Em alta 01/05/2025 16h18Atualizado01/05/2025 O...,Valor Econômico,POSITIVE,0.741187



--- Visualizando: noticias_2025-05-02_sentimento.csv ---


,url,data,titulo,texto,fonte,sentimento,score_sentimento
0,https://exame.com/mundo/eua-anunciam-desfile-m...,2025-05-02,"EUA anunciam desfile militar em 14 de junho, a...",Agência de notícias Publicado em2 de maio de 2...,Exame,NEUTRAL,0.540413
1,https://exame.com/pop/morre-a-atriz-mirim-mill...,2025-05-02,"Morre a atriz mirim Millena Brandão, após desc...",Millena Brandão: atriz é conhecida por partici...,Exame,NEGATIVE,0.775832
2,https://exame.com/esferabrasil/por-que-o-indic...,2025-05-02,Por que o índice pluviométrico é medido em mil...,São Paulo registra uma média anual de precipit...,Exame,NEGATIVE,0.534363
3,https://exame.com/brasil/com-saida-de-lupi-lul...,2025-05-02,"Com saída de Lupi, Lula faz 11ª troca minister...",Agência de notícias Publicado em2 de maio de 2...,Exame,NEGATIVE,0.701422
4,https://exame.com/tecnologia/quer-criar-grupos...,2025-05-02,Quer criar grupos no WhatsApp sem adicionar me...,Repórter Publicado em2 de maio de 2025 às19h52...,Exame,NEUTRAL,0.651505
...,...,...,...,...,...,...,...
77,https://valor.globo.com/politica/noticia/2025/...,2025-05-02,Análise: Queda de Lupi não blinda o governo,02/05/2025 19h07Atualizado02/05/2025 Aqueda de...,Valor Econômico,NEGATIVE,0.816468
78,https://valor.globo.com/financas/criptomoedas/...,2025-05-02,Bitcoin avança para US$ 97 mil com retomada de...,02/05/2025 13h04Atualizado02/05/2025 O bitcoin...,Valor Econômico,POSITIVE,0.828292
79,https://valor.globo.com/empresas/noticia/2025/...,2025-05-02,Jeff Bezos planeja vender 25 milhões de ações ...,Por Dow Jones— Nova York 02/05/2025 12h00Atual...,Valor Econômico,NEUTRAL,0.509929
80,https://valor.globo.com/empresas/noticia/2025/...,2025-05-02,"Seis anos depois, construtora Odebrecht volta ...",Por Folhapress— São Paulo 02/05/2025 19h32Atua...,Valor Econômico,NEUTRAL,0.489370



--- Visualizando: noticias_2025-05-03_sentimento.csv ---


...

,url,data,titulo,texto,fonte,sentimento,score_sentimento
0,https://exame.com/mundo/trump-promete-ordem-ex...,2025-05-11,Trump promete ordem executiva para reduzir pre...,Redação Exame Publicado em11 de maio de 2025 à...,Exame,NEGATIVE,0.798861
1,https://exame.com/marketing/gigante-chinesa-de...,2025-05-11,"Gigante chinesa de delivery, Meituan chega ao ...",(Barcroft Media / Colaborador/Getty Images) Ed...,Exame,POSITIVE,0.822726
2,https://exame.com/mundo/eua-e-china-anunciam-i...,2025-05-11,EUA e China anunciam importantes avanços em ne...,Agência de notícias Publicado em11 de maio de ...,Exame,NEGATIVE,0.628906
3,https://exame.com/marketing/com-recorde-de-pub...,2025-05-11,"Com recorde de público, Web Summit Rio segue a...",Eduardo Paes durante a abertura do Web Summit ...,Exame,POSITIVE,0.704939
4,https://exame.com/mundo/israel-diz-que-guerra-...,2025-05-11,"Israel diz que guerra em Gaza vai continuar, a...",(Ohad Zwigenberg/AFP) Agência de notícias Publ...,Exame,NEGATIVE,0.828363
5,https://exame.com/invest/mercados/openai-e-mic...,2025-05-11,OpenAI e Microsoft renegociam parceria para vi...,"Sam Altman, CEO da OpenAI, e o vice-presidente...",Exame,NEUTRAL,0.522328
6,https://exame.com/marketing/mcdonalds-traz-gri...,2025-05-11,McDonald’s traz Grimace Shake de volta em nova...,McDonald’s traz Grimace de volta e escala Rays...,Exame,POSITIVE,0.648840
7,https://exame.com/mundo/hamas-anuncia-que-vai-...,2025-05-11,Hamas anuncia que vai libertar refém israelens...,(AFP Photo) Redação Exame Publicado em11 de ma...,Exame,NEGATIVE,0.708961
8,https://exame.com/mundo/vice-premie-chines-sau...,2025-05-11,Vice-premiê chinês saúda 'avanços importantes'...,Agência de notícias Publicado em11 de maio de ...,Exame,POSITIVE,0.475590
9,https://exame.com/mundo/trump-cogita-aceitar-b...,2025-05-11,Trump cogita aceitar Boeing de luxo como prese...,Agência de notícias Publicado em11 de maio de ...,Exame,NEUTRAL,0.644289



--- Visualizando: noticias_2025-05-12_sentimento.csv ---


,url,data,titulo,texto,fonte,sentimento,score_sentimento
0,https://exame.com/brasil/precisamos-enfrentar-...,2025-05-12,Precisamos enfrentar problemas reais e não ape...,Leite: governador diz que tem apetite para lid...,Exame,NEUTRAL,0.518548
1,https://exame.com/invest/mercados/lucro-liquid...,2025-05-12,Lucro líquido da Petrobras chega a R$ 35 bi e ...,Petrobras: empresa aumentou investimentos na c...,Exame,POSITIVE,0.759656
2,https://exame.com/ciencia/nova-york-esta-afund...,2025-05-12,"Nova York está afundando, mostram imagens de s...",Dados de satélites mostram que cidades como No...,Exame,NEGATIVE,0.851781
3,https://exame.com/ciencia/tchau-cansaco-5-estr...,2025-05-12,"Tchau, cansaço: 5 estratégias comprovadas pela...",Saiba como combater a fadiga mental com estrat...,Exame,NEUTRAL,0.595187
4,https://exame.com/invest/mercados/petrobras-pe...,2025-05-12,"Petrobras (PETR3/ PETR4) vai pagar R$ 11,72 bi...",Petrobras: dividendos à vista (Petrobras/Divul...,Exame,NEUTRAL,0.654817
...,...,...,...,...,...,...,...
74,https://valor.globo.com/empresas/noticia/2025/...,2025-05-12,"Atrasada após denúncias, fábrica da BYD no Bra...",Por Reuters— São Paulo 12/05/2025 16h39Atualiz...,Valor Econômico,NEUTRAL,0.375086
75,https://valor.globo.com/empresas/marketing/not...,2025-05-12,WMcCann contrata ex-diretor de estratégia da Fbiz,12/05/2025 11h58Atualizado12/05/2025 AWMcCanna...,Valor Econômico,NEUTRAL,0.675122
76,https://valor.globo.com/empresas/noticia/2025/...,2025-05-12,Queda nas vendas leva indústria de caminhões a...,12/05/2025 11h25Atualizado12/05/2025 Depois de...,Valor Econômico,NEGATIVE,0.616651
77,https://valor.globo.com/politica/noticia/2025/...,2025-05-12,Alexandre de Moraes nega pedido da defesa de Z...,12/05/2025 19h27Atualizado12/05/2025 O ministr...,Valor Econômico,NEGATIVE,0.850891



--- Visualizando: noticias_2025-05-13_sentimento.csv ---


,url,data,titulo,texto,fonte,sentimento,score_sentimento
0,https://exame.com/brasil/governo-lula-paralisa...,2025-05-13,Governo Lula paralisa cessão da área da favela...,Agência de notícias Publicado em13 de maio de ...,Exame,NEGATIVE,0.842859
1,https://exame.com/pop/pepe-mujica-veja-livros-...,2025-05-13,'Pepe' Mujica: conheça livros sobre o ex-presi...,Encontro entre Mujica e Chomsky foi frutífero ...,Exame,NEUTRAL,0.635442
2,https://exame.com/invest/minhas-financas/proco...,2025-05-13,Procon-SP notifica Nintendo por cláusulas abus...,"NEW YORK, NY - MARCH 3: A person dressed as th...",Exame,NEUTRAL,0.502261
3,https://exame.com/mundo/houthis-pedem-que-trum...,2025-05-13,Houthis pedem que Trump pare de apoiar Israel ...,Agência de Notícias Publicado em13 de maio de ...,Exame,NEGATIVE,0.836961
4,https://exame.com/brasil/prefeito-de-sorocaba-...,2025-05-13,Prefeito de Sorocaba (SP) vira réu por suspeit...,"Prefeito 'tiktoker de Sorocaba', Rodrigo Manga...",Exame,NEGATIVE,0.847791
...,...,...,...,...,...,...,...
69,https://valor.globo.com/financas/noticia/2025/...,2025-05-13,"Dólar tem queda forte e vai a R$ 5,60, no meno...",Em alta 13/05/2025 17h08Atualizado13/05/2025 O...,Valor Econômico,NEGATIVE,0.751696
70,https://valor.globo.com/mundo/noticia/2025/05/...,2025-05-13,EUA sancionam empresas acusadas de enviar petr...,13/05/2025 17h19Atualizado13/05/2025 ODepartam...,Valor Econômico,NEGATIVE,0.851579
71,https://valor.globo.com/financas/noticia/2025/...,2025-05-13,Ibovespa dispara e bate recorde com perspectiv...,Em alta 13/05/2025 17h32Atualizado13/05/2025 A...,Valor Econômico,POSITIVE,0.767525
72,https://valor.globo.com/empresas/noticia/2025/...,2025-05-13,Redução de custos não vai impactar planos para...,13/05/2025 17h19Atualizado13/05/2025 Apresiden...,Valor Econômico,POSITIVE,0.470969
